### Options:
* BalancingStrategy = 'OverSampling'/'ClassWeights'/'None'
    * *Compensate for dataset class imbalance*
* DataAugmentation = 'True'/'False'
    * *To augment data sample while training, to avoid overfit*
* BatchSize = 16/32/64/128
    * *Size of each batch getting fed to the network*
* NumberOfEpochs = 1-*
    * *Number of local epochs on clients*
* logsPerEpoch = 1-*
    * *Amount of log steps per epoch*

In [ ]:
!python3 'vitrolife_centralized.py' --ExperimentName="Embryo-Centralized-B16-E100-No-LrSchedule" \
                                    --BalancingStrategy="None" \
                                    --DataAugmentation="False" \
                                    --BatchSize=16 \
                                    --NumberOfEpochs=100 \
                                    --LogsPerEpoch=1

In [ ]:
import sys
sys.path.append('../..')
import numpy as np

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader_oversampler as Dataloader
import utils.dataloaders.vitrolife_dataloader as DataloaderBla

path =  "/mnt/data/vitroLifeDataset"
#training_vitrolife_dataloader = Dataloader.VitroLifeDataloader(path, 32, train=True)
training_vitrolife_dataloader = DataloaderBla.VitroLifeDataloader(path, 32, train=True)
grounds = training_vitrolife_dataloader.get_ground_truths()
print(len(grounds))

batch_grounds = []
for i in range(training_vitrolife_dataloader.__len__()):
    x, y = training_vitrolife_dataloader.__getitem__(i)
    
    batch_grounds.extend(y)
    
print(len(batch_grounds))

### Apply test set on best model - Centralized eval

In [2]:
from tensorflow import keras
from keras.datasets import mnist
import numpy
import sys
sys.path.append('../..')

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as DataloaderBla

path =  "/mnt/data/vitroLifeDataset"
test_vitrolife_dataloader = DataloaderBla.VitroLifeDataloader(path, 16, dataset_split="Test")

#load model
#model_name = "../vitrolifeFederatedDynamicAugment/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DynamicDA_V2/R-114"
model_name= "../VitrolifeQFed/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DA-QFed-High-param/R-300"
#model_name= "../vitrolifeFederated/models/Embryo-Federated-B16-R100-E1-HIGH-LR-OverSampling-DA/R-87"
#model_name= "../vitrolifeCentralized/models/Embryo-Centralized-B16-E100/Epoch-100"
#model_name= "./models/Embryo-Centralized-B16-E100-OverSampling-DA/Epoch-100"
model_from_disk = keras.models.load_model(f"{model_name}")

result = model_from_disk.evaluate(test_vitrolife_dataloader)

print(f"Experiment model: {model_name}")
print(f"loss={result[0]}\n tp={result[1]}\n fp={result[2]}\n tn={result[3]}\n fn={result[4]}\n accuracy={result[5]}\n presision={result[6]}\n recall={result[7]}\n auc={result[8]}\n prc={result[9]}")

207/207 [==============================] - 31s 144ms/step - loss: 0.6787 - tp: 788.0000 - fp: 1113.0000 - tn: 1110.0000 - fn: 301.0000 - accuracy: 0.5731 - precision: 0.4145 - recall: 0.7236 - auc: 0.6363 - prc: 0.4321
Experiment model: ../VitrolifeQFed/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DA-QFed-High-param/R-300
loss=0.6787009835243225
 tp=788.0
 fp=1113.0
 tn=1110.0
 fn=301.0
 accuracy=0.5730676054954529
 presision=0.41451868414878845
 recall=0.7235996127128601
 auc=0.6363219022750854
 prc=0.43209293484687805


### Apply test set - Federated eval

In [3]:
from tensorflow import keras
from keras.datasets import mnist
import numpy as np
import sys
sys.path.append('../..')

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as DataloaderBla


#load model
#model_name= "../vitrolifeCentralized/models/Embryo-Centralized-B16-E100/Epoch-100"
#model_name= "./models/Embryo-Centralized-B16-E100-OverSampling-DA/Epoch-100"
#model_name= "../vitrolifeFederated/models/Embryo-Federated-B16-R100-E1-HIGH-LR-OverSampling-DA/R-87"
model_name = "../vitrolifeFederatedDynamicAugment/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DynamicDA_V2/R-114"
#model_name= "../VitrolifeQFed/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DA-QFed-High-param/R-300"
model_from_disk = keras.models.load_model(f"{model_name}")

all_results = []
batch_size = 16
number_of_clients=23
for client_id in range(23):
    path =  "/mnt/data/vitroLifeDataset"
    test_loader = DataloaderBla.VitroLifeDataloader(path, batch_size, dataset_split="Test", clinic_ID=client_id)
    result = model_from_disk.evaluate(test_loader)
    #if result[8] >= 0.50:
    all_results.append(result)
    #else:
    #    print("skipping low")
    #    number_of_clients = number_of_clients -1


# acum all 23 clients metrics
aggregated_results = [0,0,0,0,0,0,0,0,0,0]    
for partial_results in all_results:
    for index, metric in enumerate(partial_results):
            aggregated_results[index] += metric
        
        
# take mean        
for index, agg_res in enumerate(aggregated_results):
    aggregated_results[index] = aggregated_results[index]/number_of_clients
    
     
    
standart_deviation = []    
# collect std    
for i in range(10):
    metrics_groups = [item[i] for item in all_results]
    standart_deviation.append(np.std(metrics_groups))

print("std")
print(standart_deviation)
    

metric_names =  ['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'presision', 'recall', 'auc', 'prc']
for i, final_res in enumerate(aggregated_results):
    print(f"{metric_names[i]} {final_res} +/- {standart_deviation[i]}")
    

#print(f"Experiment model: {model_name}")
#print(f"loss={result[0]}\n tp={result[1]}\n fp={result[2]}\n tn={result[3]}\n fn={result[4]}\n accuracy={result[5]}\n presision={result[6]}\n recall={result[7]}\n auc={result[8]}\n prc={result[9]}")

2/2 [==============================] - 0s 136ms/step - loss: 0.7451 - tp: 4.0000 - fp: 13.0000 - tn: 13.0000 - fn: 2.0000 - accuracy: 0.5312 - precision: 0.2353 - recall: 0.6667 - auc: 0.5513 - prc: 0.2273
std
[0.05802864983634612, 34.018842771559235, 53.40978295157019, 71.94455437065186, 16.60334537088011, 0.06759684582986006, 0.11179528539519974, 0.1450486553138808, 0.11526542059050644, 0.12974720827403052]
loss 0.680362457814424 +/- 0.05802864983634612
tp 30.391304347826086 +/- 34.018842771559235
fp 42.21739130434783 +/- 53.40978295157019
tn 50.26086956521739 +/- 71.94455437065186
fn 14.73913043478261 +/- 16.60334537088011
accuracy 0.5866532766300699 +/- 0.06759684582986006
presision 0.42669915375502215 +/- 0.11179528539519974
recall 0.7024781431840814 +/- 0.1450486553138808
auc 0.6258825037790381 +/- 0.11526542059050644
prc 0.4447702448005262 +/- 0.12974720827403052


### Remove trained clients with less than 150 training samples 

In [4]:
from tensorflow import keras
from keras.datasets import mnist
import numpy as np
import sys
sys.path.append('../..')

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as DataloaderBla


#load model
#model_name= "../vitrolifeCentralized/models/Embryo-Centralized-B16-E100/Epoch-100"
#model_name= "./models/Embryo-Centralized-B16-E100-OverSampling-DA/Epoch-100"
#model_name= "../vitrolifeFederated/models/Embryo-Federated-B16-R100-E1-HIGH-LR-OverSampling-DA/R-87"
model_name = "../vitrolifeFederatedDynamicAugment/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DynamicDA_V2/R-114"
#model_name= "../VitrolifeQFed/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DA-QFed-High-param/R-300"
model_from_disk = keras.models.load_model(f"{model_name}")

all_results = []
batch_size = 16
number_of_clients=23
for client_id in range(23):
    path =  "/mnt/data/vitroLifeDataset"
    test_loader = DataloaderBla.VitroLifeDataloader(path, batch_size, dataset_split="Test", clinic_ID=client_id)
    train_loader = DataloaderBla.VitroLifeDataloader(path, batch_size, dataset_split="Train", clinic_ID=client_id)
    result = model_from_disk.evaluate(test_loader)
    if train_loader.count() >= 150:
        all_results.append(result)
    else:
        print("skipping low")
        number_of_clients = number_of_clients -1


# acum all 23 clients metrics
aggregated_results = [0,0,0,0,0,0,0,0,0,0]    
for partial_results in all_results:
    for index, metric in enumerate(partial_results):
            aggregated_results[index] += metric
        
        
# take mean        
for index, agg_res in enumerate(aggregated_results):
    aggregated_results[index] = aggregated_results[index]/number_of_clients
    
     
    
standart_deviation = []    
# collect std    
for i in range(10):
    metrics_groups = [item[i] for item in all_results]
    standart_deviation.append(np.std(metrics_groups))

print("std")
print(standart_deviation)
    

metric_names =  ['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'presision', 'recall', 'auc', 'prc']
for i, final_res in enumerate(aggregated_results):
    print(f"{metric_names[i]} {final_res} +/- {standart_deviation[i]}")
    

#print(f"Experiment model: {model_name}")
#print(f"loss={result[0]}\n tp={result[1]}\n fp={result[2]}\n tn={result[3]}\n fn={result[4]}\n accuracy={result[5]}\n presision={result[6]}\n recall={result[7]}\n auc={result[8]}\n prc={result[9]}")

1/1 [==============================] - 0s 160ms/step - loss: 0.8055 - tp: 2.0000 - fp: 2.0000 - tn: 4.0000 - fn: 3.0000 - accuracy: 0.5455 - precision: 0.5000 - recall: 0.4000 - auc: 0.4333 - prc: 0.5532
skipping low
1/1 [==============================] - 0s 194ms/step - loss: 0.7849 - tp: 3.0000 - fp: 7.0000 - tn: 4.0000 - fn: 1.0000 - accuracy: 0.4667 - precision: 0.3000 - recall: 0.7500 - auc: 0.3636 - prc: 0.2028
skipping low
1/1 [==============================] - 0s 165ms/step - loss: 0.6426 - tp: 3.0000 - fp: 3.0000 - tn: 5.0000 - fn: 0.0000e+00 - accuracy: 0.7273 - precision: 0.5000 - recall: 1.0000 - auc: 0.7500 - prc: 0.3948
skipping low
1/1 [==============================] - 0s 121ms/step - loss: 0.5659 - tp: 1.0000 - fp: 2.0000 - tn: 4.0000 - fn: 1.0000 - accuracy: 0.6250 - precision: 0.3333 - recall: 0.5000 - auc: 0.7500 - prc: 0.3657
skipping low
1/1 [==============================] - 0s 208ms/step - loss: 0.5977 - tp: 8.0000 - fp: 5.0000 - tn: 3.0000 - fn: 0.0000e+00 - ac

### PLOT CM 

In [ ]:
from tensorflow import keras
from keras.datasets import mnist
import wandb
import numpy
import sys
sys.path.append('../..')

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as DataloaderBla

project_name = "Embryo-eval"
wandb.init(project=f"{project_name}", entity="anton-peter")


wandb.run.name = "DynamicAggresiveDA-FedAVG/R-114"

path =  "/mnt/data/vitroLifeDataset"
test_vitrolife_dataloader = DataloaderBla.VitroLifeDataloader(path, 16, dataset_split="Test")

classes = ['No Pregnancy', 'Pregnancy']
        
#load model

#model_name= "../vitrolifeCentralized/models/Embryo-Centralized-B16-E100/Epoch-100"
#model_name= "./models/Embryo-Centralized-B16-E100-OverSampling-DA/Epoch-100"
model_name = "../vitrolifeFederatedDynamicAugment/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DynamicDA_V2/R-114"
#model_name= "../VitrolifeQFed/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DA-QFed-High-param/R-300"
#model_name= "../vitrolifeFederated/models/Embryo-Federated-B16-R100-E1-HIGH-LR-OverSampling-DA/R-87"
model_from_disk = keras.models.load_model(f"{model_name}")

ground_truths = test_vitrolife_dataloader.get_ground_truths()
predicted_probabilities = model_from_disk.predict(test_vitrolife_dataloader)

predicted_arg_max = []
predicted_probabilities_transformed = []

for proba in predicted_probabilities:
    predicted_probabilities_transformed.append([1-proba, proba])
    if proba > 0.5:
        predicted_arg_max.append(1)
    else:
        predicted_arg_max.append(0)


wandb.log({"roc" : wandb.plot.roc_curve(ground_truths, predicted_probabilities_transformed, labels=classes, classes_to_plot=[1])})
wandb.sklearn.plot_confusion_matrix(ground_truths, predicted_arg_max, labels=classes)
wandb.finish()

### PLOT CM Federated

In [ ]:
from tensorflow import keras
from keras.datasets import mnist
import wandb
import numpy
import sys
sys.path.append('../..')

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as DataloaderBla

project_name = "Embryo-eval"
wandb.init(project=f"{project_name}", entity="anton-peter")
wandb.run.name = "Embryo-Federated-B16-R100-E1-HIGH-LR-OverSampling-DA/R-88"

model_name= "../vitrolifeFederated/models/Embryo-Federated-B16-R100-E1-HIGH-LR-OverSampling-DA/R-88"
model_from_disk = keras.models.load_model(f"{model_name}")

ground_truths = []
predicted = []

ground_truths_matrix = []
predicted_matrix = []

for client_id in range(23):
    path =  "/mnt/data/vitroLifeDataset"
    test_loader = DataloaderBla.VitroLifeDataloader(path, batch_size, dataset_split="Test", clinic_ID=client_id)
    result = model_from_disk.evaluate(test_loader)
    current_predicted =  model_from_disk.predict(test_loader)
    current_ground = test_loader.get_ground_truths()
    print("Loading done")

    # Be aware of max sampling warning
    current_ground = current_ground
    current_predicted = current_predicted.tolist()

    ground_truths_matrix.extend(current_ground)
    predicted_matrix.extend(current_predicted)

    ground_truths.extend(current_ground)
    predicted.extend(current_predicted)               

predicted_arg_max = []
predicted_probabilities_transformed = []

# For binary classification
for proba in predicted:
    proba = proba[0]
    predicted_probabilities_transformed.append([1-proba, proba])
    if proba > 0.5:
        predicted_arg_max.append(1)
    else:
        predicted_arg_max.append(0)

predicted_arg_max = [proba.index(max(proba)) for proba in predicted_matrix]
classes = ['No Pregnancy', 'Pregnancy']       

wandb.log({"roc" : wandb.plot.roc_curve(ground_truths, predicted_probabilities_transformed, labels=None)})
wandb.sklearn.plot_confusion_matrix(ground_truths, predicted_arg_max, labels=classes)
wandb.finish()               

### Test from disk

In [ ]:
import numpy as np
from sklearn import metrics
from tensorflow import keras
import sys
sys.path.append('../..')
import numpy as np

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as Dataloader


classes = ['No Pregnancy', 'Pregnancy']

path =  "/mnt/data/vitroLifeDataset"
testing_vitrolife_dataloader = Dataloader.VitroLifeDataloader(path, 32, train=False)
    
ground_truths = testing_vitrolife_dataloader.sample_labels[0:testing_vitrolife_dataloader.__len__()*32]

model = keras.models.load_model('./models/model-Test3')

predicted_probabilities = model.predict(testing_vitrolife_dataloader)

auc = metrics.roc_auc_score(ground_truths, predicted_probabilities)
print(auc)

In [ ]:
uc = metrics.roc_auc_score(ground_truths, predicted_probabilities)
print(auc)
import numpy as np
from sklearn import metrics
from tensorflow import keras
import sys
sys.path.append('../..')
import numpy as np
import wandb

import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as Dataloader

project_name = "Test"
wandb.init(project=f"{project_name}", entity="anton-peter")
wandb.run.name = "test"


classes = ['No Pregnancy', 'Pregnancy']


ground_truths = [1, 1,0,0,1,0,1,1,0,1] 
predicted_probabilities = [0.51, 0.4, 0.3, 0.15, 0.6, 0.3, 0.51, 0.6, 0.3, 0.7]


print(len(ground_truths))
print(len(predicted_probabilities))
predicted_arg_max = []
predicted_probabilities_transformed = []

for proba in predicted_probabilities:
    predicted_probabilities_transformed.append([1-proba, proba])
    if proba > 0.5:
        predicted_arg_max.append(1)
    else:
        predicted_arg_max.append(0)


wandb.log({"roc" : wandb.plot.roc_curve(ground_truths, predicted_probabilities_transformed, labels=classes)})


wandb.sklearn.plot_confusion_matrix(ground_truths, predicted_arg_max, labels=classes)
    

In [ ]:
import random

random.randint(-45,45)

In [ ]:
import keras.backend as K
from tensorflow import keras
import numpy
import sys
sys.path.append('../..')


# 0.0000001
#model_name = "Embryo-Centralized-B16-E100-No-LrSchedule/Epoch-97"
model_name = "Embryo-Centralized-B16-E100/Epoch-100"
model_from_disk = keras.models.load_model(f"models/{model_name}")
print(K.eval(model_from_disk.optimizer.lr))



### Visualize filters

In [ ]:
from tensorflow import keras
from keras import Model
import matplotlib.pyplot as plt
from keras.datasets import mnist
import numpy as np
import sys
sys.path.append('../..')

import utils.dataloaders.vitrolife_dataloader as Dataloader


model_name= "Embryo-Centralized-B16-E100/Epoch-99"
model_from_disk = keras.models.load_model(f"models/{model_name}")
model_from_disk.summary()

conv_2d_layers = [0,4,9,4]

outputs = [model_from_disk.layers[i].output for i in conv_2d_layers]
model_truncated = Model(inputs=model_from_disk.inputs, outputs=outputs)

model_truncated.summary()

path_to_data = "/mnt/data/vitroLifeDataset"
testing_vitrolife_dataloader = Dataloader.VitroLifeDataloader(path_to_data, 16, dataset_split="Test")

batch = testing_vitrolife_dataloader.__getitem__(0)
print(batch[1][1])
raw_img  = batch[0][1]

raw_img = np.expand_dims(raw_img, axis=0)

print(raw_img.shape)

feature_map = model_truncated.predict(raw_img)


cols = 6
rows = 6

for ftr in feature_map:
    fig = plt.figure(figsize=(12,12))
    for i in range(1, 7):
        fig = plt.subplot(rows,cols,i)
        fig.set_xticks([])
        fig.set_yticks([])
        plt.imshow(ftr[0,:,:,i], cmap='gray')
    
    plt.show()
